# General Guide for Parallelizing xCDAT Operations with Dask

Author: [Tom Vo](https://github.com/tomvothecoder)

Date: 05/22/23


## Overview

This notebook serves as a general guide for parallel computing with xCDAT. It covers the
following topics:

- Basics of Dask Arrays
- General Dask Best Practice
- How to use Dask with Xarray
- How to use Dask with xCDAT, including real-world examples and performance metrics.

_The data used in the code examples can be found through the [Earth System Grid Federation (ESGF) search portal](https://aims2.llnl.gov/search)._


## Notebook Setup

Create an Anaconda environment for this notebook using the command below:

```bash
conda create -n xcdat_dask_guide -c conda-forge xarray dask xcdat flox matplotlib nc-time-axis jupyter
```

- [`flox`](https://flox.readthedocs.io/en/latest/) is a package that is used to improve the xarray `groupby()` performance by
  making it parallelizable.
- `matplotlib` is an optional dependency required for plotting with xarray
- [`nc-time-axis`](https://nc-time-axis.readthedocs.io/en/latest/) is an optional dependency required for `matplotlib` to plot `cftime` coordinates


## Basics of Dask Arrays

- **Dask divides arrays** into many small pieces, called **"chunks"** (each presumed to be small enough to fit into memory)
- Dask Array **operations are lazy**
  - Operations **queue** up a series of tasks mapped over blocks
  - No computation is performed until values need to be computed (hence "lazy")
  - Data is loaded into memory and **computation** is performed in **streaming fashion**, **block-by-block**
- Computation is controlled by multi-processing or thread pool

<div style="text-align:center">
  <img src="../_static/dask-array.png" alt="Dask Array" style="display: inline-block; width:300px;">
</div>

Source: <cite>https://docs.xarray.dev/en/stable/user-guide/dask.html</cite>


## General Dask Best Practices

- [Use NumPy](https://docs.dask.org/en/stable/array-best-practices.html#use-numpy)
  - If your **data fits comfortably in RAM and you are not performance bound**, then using **NumPy might be the right choice**. Dask adds another layer of complexity which may get in the way.
  - If you are just looking for **speedups rather than scalability** then you may want to **consider a project like [Numba](https://numba.pydata.org/)**
- [Select a good chunk size](https://docs.dask.org/en/stable/array-best-practices.html#select-a-good-chunk-size)
  - A **common performance problem among Dask Array users** is that they have chosen a **chunk size** that is either **too small** (leading to lots of overhead) or **poorly aligned with their data** (leading to inefficient reading).
- [Orient your chunks](https://docs.dask.org/en/stable/array-best-practices.html#orient-your-chunks)
  - When reading data you should **align your chunks with your storage format**.
- [Avoid Oversubscribing Threads](https://docs.dask.org/en/stable/array-best-practices.html#avoid-oversubscribing-threads)
  - **By default Dask will run as many concurrent tasks as you have logical cores.** It assumes that **each task** will consume about **one core**. However, many array-computing libraries are themselves multi-threaded, which can cause contention and low performance.
- [Consider Xarray](hhttps://docs.dask.org/en/stable/array-best-practices.html#consider-xarray)
  - The **Xarray package wraps** around **Dask Array**, and so offers the **same scalability**, but also adds **convenience when dealing with complex datasets**.
- [Build your own Operations](https://docs.dask.org/en/stable/array-best-practices.html#build-your-own-operations)
  - Often we want to perform computations for which there is no exact function in Dask Array. In these cases we may be able to use some of the more generic functions to build our own.

Source: <cite>https://docs.dask.org/en/stable/array-best-practices.html#best-practices</cite>


## How to use Dask with Xarray


### Quick Overview


- Why does Xarray integrate with Dask?
  > Xarray integrates with Dask to support parallel computations and streaming computation
  > on datasets that don’t fit into memory. Currently, Dask is an entirely optional feature
  > for xarray. However, the benefits of using Dask are sufficiently strong that Dask may
  > become a required dependency in a future version of xarray.
  >
  > &mdash; <cite>https://docs.xarray.dev/en/stable/use
- Which Xarray features support Dask?

  > Nearly all existing xarray methods (including those for indexing, computation,
  > concatenating and grouped operations) have been extended to work automatically with
  > Dask arrays. When you load data as a Dask array in an xarray data structure, almost
  > all xarray operations will keep it as a Dask array; when this is not possible, they
  > will raise an exception rather than unexpectedly loading data into memory.
  >
  > &mdash; <cite>https://docs.xarray.dev/en/stable/user-guide/dask.html#using-dask-with-xarray</cite>

- What is the default Dask behavior for distributing work on compute hardware?

  > By default, dask uses its multi-threaded scheduler, which distributes work across
  > multiple cores and allows for processing some datasets that do not fit into memory.
  > For running across a cluster, [setup the distributed scheduler](https://docs.dask.org/en/latest/setup.html).
  >
  > &mdash; <cite>https://docs.xarray.dev/en/stable/user-guide/dask.html#using-dask-with-xarray</cite>

- How do I use Dask arrays in an `xarray.Dataset`?

  > The usual way to create a Dataset filled with Dask arrays is to load the data from a
  > netCDF file or files. You can do this by supplying a `chunks` argument to [open_dataset()](https://docs.xarray.dev/en/stable/generated/xarray.open_dataset.html#xarray.open_dataset)
  > or using the [open_mfdataset()](https://docs.xarray.dev/en/stable/generated/xarray.open_mfdataset.html#xarray.open_mfdataset) function.

- What happens if I don't specify `chunks` with `open_mfdataset()`?

  > `open_mfdataset()` called without `chunks` argument will return dask arrays with
  > chunk sizes equal to the individual files. Re-chunking the dataset after creation
  > with `ds.chunk()` will lead to an ineffective use of memory and is not recommended.
  >
  > &mdash; <cite>https://docs.xarray.dev/en/stable/user-guide/dask.html#reading-and-writing-data</cite>

- Are there any optimizations tips for working with Dask and Xarray?

  - We HIGHLY recommend checking out the [Optimization Tips](https://docs.xarray.dev/en/stable/user-guide/dask.html#optimization-tips) section if you are using Dask with Xarray.


### Chunking Best Practices


> For performance, a good choice of `chunks` follows the following rules:
>
> 1. A chunk should be small enough to fit comfortably in memory. We'll
>    have many chunks in memory at once
> 2. A chunk must be large enough so that computations on that chunk take
>    significantly longer than the 1ms overhead per task that Dask scheduling
>    incurs. A task should take longer than 100ms
> 3. Chunk sizes between 10MB-1GB are common, depending on the availability of
>    RAM and the duration of computations
> 4. Chunks should align with the computation that you want to do.
>    - For example, if you plan to frequently slice along a particular dimension,
>      then it's more efficient if your chunks are aligned so that you have to
>      touch fewer chunks. If you want to add two arrays, then its convenient if
>      those arrays have matching chunks patterns
> 5. Chunks should align with your storage, if applicable.
>    - Array data formats are often chunked as well. When loading or saving data,
>      if is useful to have Dask array chunks that are aligned with the chunking
>      of your storage, often an even multiple times larger in each direction
>
> &mdash; <cite>https://docs.dask.org/en/latest/array-chunks.html</cite>


### Chunking and Performance


> The chunks parameter has critical performance implications when using Dask arrays. If
> your chunks are too small, queueing up operations will be extremely slow, because Dask
> will translate each operation into a huge number of operations mapped across chunks.
> Computation on Dask arrays with small chunks can also be slow, because each operation
> on a chunk has some fixed overhead from the >Python interpreter and the Dask task
> executor.
>
> Conversely, if your chunks are too big, some of your computation may be wasted, because
> Dask only computes results one chunk at a time.
>
> A good rule of thumb is to create arrays with a minimum chunksize of at least one
> million elements (e.g., a 1000x1000 > matrix). With large arrays (10+ GB), the cost of
> queueing up Dask operations can be noticeable, and you may need even > larger
> chunksizes.
>
> &mdash; <cite>https://docs.xarray.dev/en/stable/user-guide/dask.html#chunking-and-performance</cite>


#### User Tip: You can let Dask figure out the optimal chunk size

Dask Arrays can look for a `.chunks` attribute and use that to provide a good chunking.
This helps prevent users from specifying "too many chunks" and "too few chunks" which
can lead to performance issues.

To do this in `xarray.open_dataset()` and `xarray.open_mfdataset()`, specify:

1. `chunks={"time": "auto"}`: auto-scale the specified dimension to get to accommodate ideal chunk sizes
   - replace `"time"` and/or add additional dims to the dictionary for auto-scaling
2. `chunks="auto"`: allow chunking _all_ dimensions to accommodate ideal chunk sizes

> &mdash; <cite>https://docs.dask.org/en/latest/array-chunks.html#automatic-chunking</cite>


### Example of Chunking with Xarray

The code example below demonstrates chunking a dataset in Xarray.

**Disclaimer: The dataset used in the example is only a few hundred MBs. A file this small
should typically NOT be chunked since computational performance will most likely suffer.**


In [9]:
import xarray as xr

filepath = "http://esgf.nci.org.au/thredds/dodsC/master/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/historical/r10i1p1f1/Amon/tas/gn/v20200605/tas_Amon_ACCESS-ESM1-5_historical_r10i1p1f1_gn_185001-201412.nc"


Case 1 - Explicitly chunk the time dimension by 10.


In [22]:
ds2 = xr.open_dataset(filepath, chunks={"time": "10"})

ds2


<xarray.Dataset>
Dimensions:    (time: 1980, bnds: 2, lat: 145, lon: 192)
Coordinates:
  * time       (time) datetime64[ns] 1850-01-16T12:00:00 ... 2014-12-16T12:00:00
  * lat        (lat) float64 -90.0 -88.75 -87.5 -86.25 ... 86.25 87.5 88.75 90.0
  * lon        (lon) float64 0.0 1.875 3.75 5.625 ... 352.5 354.4 356.2 358.1
    height     float64 ...
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(1, 2), meta=np.ndarray>
    lat_bnds   (lat, bnds) float64 dask.array<chunksize=(145, 2), meta=np.ndarray>
    lon_bnds   (lon, bnds) float64 dask.array<chunksize=(192, 2), meta=np.ndarray>
    tas        (time, lat, lon) float32 dask.array<chunksize=(1, 145, 192), meta=np.ndarray>
Attributes: (12/49)
    Conventions:                     CF-1.7 CMIP-6.2
    activity_id:                     CMIP
    branch_method:                   standard
    branch_time_in_child:            0.0
    branch_time_in_parent:           87658.0
    creation_date:                   2020-06-05T04:06:11Z
    ...                              ...
    version:                         v20200605
    license:                         CMIP6 model data produced by CSIRO is li...
    cmor_version:                    3.4.0
    _NCProperties:                   version=2,netcdf=4.6.2,hdf5=1.10.5
    tracking_id:                     hdl:21.14100/af78ae5e-f3a6-4e99-8cfe-5f2...
    DODS_EXTRA.Unlimited_Dimension:  time

Now we perform a daily average using the `groupby` API.

`flox` must be installed to make this API parallelizable and Xarray will use `flox` by
default if it is installed.


In [28]:
tas_daily_avg = (
    ds2["tas"].groupby(ds2.time.dt.day).mean(method="cohorts", engine="flox")
)
tas_daily_avg


TypeError: nanmean() got an unexpected keyword argument 'method'

Case 2 - Let Dask auto-scale all dimensions to get a good chunk size using `chunks="auto"`,
which references the `.chunks` attribute.


In [18]:
ds1 = xr.open_dataset(filepath, chunks="auto")

ds1


<xarray.Dataset>
Dimensions:    (time: 1980, bnds: 2, lat: 145, lon: 192)
Coordinates:
  * time       (time) datetime64[ns] 1850-01-16T12:00:00 ... 2014-12-16T12:00:00
  * lat        (lat) float64 -90.0 -88.75 -87.5 -86.25 ... 86.25 87.5 88.75 90.0
  * lon        (lon) float64 0.0 1.875 3.75 5.625 ... 352.5 354.4 356.2 358.1
    height     float64 ...
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(1980, 2), meta=np.ndarray>
    lat_bnds   (lat, bnds) float64 dask.array<chunksize=(145, 2), meta=np.ndarray>
    lon_bnds   (lon, bnds) float64 dask.array<chunksize=(192, 2), meta=np.ndarray>
    tas        (time, lat, lon) float32 dask.array<chunksize=(1695, 122, 162), meta=np.ndarray>
Attributes: (12/49)
    Conventions:                     CF-1.7 CMIP-6.2
    activity_id:                     CMIP
    branch_method:                   standard
    branch_time_in_child:            0.0
    branch_time_in_parent:           87658.0
    creation_date:                   2020-06-05T04:06:11Z
    ...                              ...
    version:                         v20200605
    license:                         CMIP6 model data produced by CSIRO is li...
    cmor_version:                    3.4.0
    _NCProperties:                   version=2,netcdf=4.6.2,hdf5=1.10.5
    tracking_id:                     hdl:21.14100/af78ae5e-f3a6-4e99-8cfe-5f2...
    DODS_EXTRA.Unlimited_Dimension:  time

In [25]:
ds2.chunks


Frozen({'time': (1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

### More Resources

Now that you have a high-level introduction to Dask with Xarray, here are more resources
to level up your knowledge.

- [Official Xarray Parallel Computing with Dask Guide](https://docs.xarray.dev/en/stable/user-guide/dask.html)
- [Official Xarray Parallel Computing with Dask Jupyter Notebook Tutorial](https://tutorial.xarray.dev/intermediate/xarray_and_dask.html)
- [Official Dask guide for Xarray with Dask Arrays](https://examples.dask.org/xarray.html)
- [Dask blog post: "Choosing good chunk sizes in Dask"](https://blog.dask.org/2021/11/02/choosing-dask-chunk-sizes)


## How to Use Dask with xCDAT

Many core xCDAT APIs inherit Xarray's Dask support by operating on Xarray Dataset objects and making calls to parallelized Xarray Dataset APIs.

The only thing xCDAT users have to do is **chunk the dataset** and **call any of the parallelizable xCDAT APIs**.


### Parallelizable APIs

1. [xarray.Dataset.spatial.average](../generated/xarray.Dataset.spatial.average.rst)
   - Sequential Operations:
     - generating weights using lat/lon coordinate bounds
     - swapping longitude axis orientation
     - scaling domain bounds to a specified region
   - Parallelizable Operations:
   - The averaging computation, which uses `xarray.DataArray.weighted().mean()`

- `xcdat.Dataset.temporal.<average|group_average|climatology|departures>`
  - generating weights using time coordinate bounds

2. [xarray.Dataset.temporal.average](../generated/xarray.Dataset.temporal.average.rst),
   [xarray.Dataset.temporal.group_average](../generated/xarray.Dataset.temporal.group_average.rst),
   [xarray.Dataset.temporal.climatology](../generated/xarray.Dataset.temporal.climatology.rst),
   [xarray.Dataset.temporal.departures](../generated/xarray.Dataset.temporal.departures.rst)
   - Sequential Operations:
     - generating weights using time coordinate bounds
   - Parallelizable Operations:
     - The grouping operation `xarray.Dataset.groupby()` (**must have `flox` installed**)


## Code Examples with xCDAT


### Performance Metrics


### FAQS


### Are there cases where xCDAT loads Dask arrays into memory?

xCDAT loads Dask arrays into memory when performing operations or computations using
multi-dimensional arrays, specifically coordinate bounds . As of `xarray=2023.5.0`,
`xarray can't set arrays with multiple array indices to dask yet`, so these arrays must
be loaded into memory.

xCDAT loads coordinate bounds into memory in the following APIs during specific
situations:

- `xcdat.axis.swap_lon_axis`
  - swapping longitude axis orientation
  - aligning longitude bounds to 360 axis
- `xarray.Dataset.spatial.average`
  - generating weights using lat/lon coordinate bounds
  - swapping longitude axis orientation
  - scaling domain bounds to a specified region
- `xcdat.Dataset.temporal.<average|group_average|climatology|departures>`
  - generating weights using time coordinate bounds
